# RomKorr – Data Prep

Dieses Notebook normalisiert die bestehende CSV `rom_korr_full_website_coords.csv` (kein Re-Scrape), erzeugt stabile IDs/Links und ein maschinenlesbares Datum.

Outputs:
- `data/processed/letters_master.parquet`
- `data/processed/letters_master.csv`
- `data/processed/places_agg.parquet` (Orte aggregiert für Hotspots/Heatmap)

**Design-Ziel:** Ein *einziges* "Master"-Dataset, aus dem alle Visualisierungen abgeleitet werden.

In [1]:
from __future__ import annotations

import re
from pathlib import Path
from typing import Optional

import numpy as np
import pandas as pd

In [2]:
# =========================
# Pfade
# =========================
PROJECT_ROOT = Path.cwd()

IN_CSV = PROJECT_ROOT / "rom_korr_full_website_coords.csv"

OUT_DIR = PROJECT_ROOT / "data" / "processed"
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_MASTER_PARQUET = OUT_DIR / "letters_master.parquet"
OUT_MASTER_CSV = OUT_DIR / "letters_master.csv"
OUT_PLACES_AGG = OUT_DIR / "places_agg.parquet"

In [3]:
df = pd.read_csv(IN_CSV)
df.head()

,Date,Sender,Recipient,Place of Dispatch,Place of Destination,Dispatch_GeoNames,Destination_GeoNames,link,Dispatch_Lat,Dispatch_Lon,Destination_Lat,Destination_Lon,Distance_km
0,"Donnerstag, 6. Januar 1791",August Wilhelm von Schlegel,Christian Gottlob Heyne,Göttingen,Göttingen,https://www.geonames.org/2918632,https://www.geonames.org/2918632,https://briefe-der-romantik.de/letters/view/1?...,51.53443,9.93228,51.53443,9.93228,0.000000
1,"Mittwoch, 20. September 1797",August Wilhelm von Schlegel,Christian Gottlob Heyne,Jena,Göttingen,https://www.geonames.org/2895044,https://www.geonames.org/2918632,https://briefe-der-romantik.de/letters/view/2?...,50.92878,11.58990,51.53443,9.93228,133.955266
2,"Samstag, 26. Mai 1798",August Wilhelm von Schlegel,Georg Joachim Göschen,Berlin,Leipzig,https://www.geonames.org/2950159,https://www.geonames.org/2879139,https://briefe-der-romantik.de/letters/view/3?...,52.52437,13.41053,51.33962,12.37129,149.951785
3,"Mittwoch, 31. Oktober 1798",August Wilhelm von Schlegel,Georg Joachim Göschen,Jena,Leipzig,https://www.geonames.org/2895044,https://www.geonames.org/2879139,https://briefe-der-romantik.de/letters/view/4?...,50.92878,11.58990,51.33962,12.37129,71.276442
4,[Mitte August 1801],Sophie Bernhardi,August Wilhelm von Schlegel,Berlin,Jena,https://www.geonames.org/2950159,https://www.geonames.org/2895044,https://briefe-der-romantik.de/letters/view/5?...,52.52437,13.41053,50.92878,11.58990,217.565424


## 1) Spalten vereinheitlichen

Wir halten intern **snake_case** (robuster fürs Programmieren) und behalten die Original-Spalten bei Bedarf als Export.

In [4]:
RENAME = {
    "Place of Dispatch": "place_dispatch",
    "Place of Destination": "place_destination",
    "Dispatch_GeoNames": "dispatch_geonames_url",
    "Destination_GeoNames": "destination_geonames_url",
    "Dispatch_Lat": "dispatch_lat",
    "Dispatch_Lon": "dispatch_lon",
    "Destination_Lat": "destination_lat",
    "Destination_Lon": "destination_lon",
    "Distance_km": "distance_km",
}

df = df.rename(columns={k: v for k, v in RENAME.items() if k in df.columns})

# Safety: expected columns
for col in ["Sender", "Recipient", "Date", "place_dispatch", "place_destination", "link"]:
    if col not in df.columns:
        df[col] = pd.NA

df.head()

,Date,Sender,Recipient,place_dispatch,place_destination,dispatch_geonames_url,destination_geonames_url,link,dispatch_lat,dispatch_lon,destination_lat,destination_lon,distance_km
0,"Donnerstag, 6. Januar 1791",August Wilhelm von Schlegel,Christian Gottlob Heyne,Göttingen,Göttingen,https://www.geonames.org/2918632,https://www.geonames.org/2918632,https://briefe-der-romantik.de/letters/view/1?...,51.53443,9.93228,51.53443,9.93228,0.000000
1,"Mittwoch, 20. September 1797",August Wilhelm von Schlegel,Christian Gottlob Heyne,Jena,Göttingen,https://www.geonames.org/2895044,https://www.geonames.org/2918632,https://briefe-der-romantik.de/letters/view/2?...,50.92878,11.58990,51.53443,9.93228,133.955266
2,"Samstag, 26. Mai 1798",August Wilhelm von Schlegel,Georg Joachim Göschen,Berlin,Leipzig,https://www.geonames.org/2950159,https://www.geonames.org/2879139,https://briefe-der-romantik.de/letters/view/3?...,52.52437,13.41053,51.33962,12.37129,149.951785
3,"Mittwoch, 31. Oktober 1798",August Wilhelm von Schlegel,Georg Joachim Göschen,Jena,Leipzig,https://www.geonames.org/2895044,https://www.geonames.org/2879139,https://briefe-der-romantik.de/letters/view/4?...,50.92878,11.58990,51.33962,12.37129,71.276442
4,[Mitte August 1801],Sophie Bernhardi,August Wilhelm von Schlegel,Berlin,Jena,https://www.geonames.org/2950159,https://www.geonames.org/2895044,https://briefe-der-romantik.de/letters/view/5?...,52.52437,13.41053,50.92878,11.58990,217.565424


## 2) Stabile `letter_id` & kanonischer Link

In deinen `link`-URLs steckt (teilweise) ein `query_id=...`. Das ist für Langzeit-Reproduzierbarkeit ungeeignet.

Wir extrahieren daher `letter_id` aus `/letters/view/<id>` und bauen eine kanonische URL ohne `query_id`.

In [5]:
BASE_CANONICAL = "https://briefe-der-romantik.de/letters/view/"

def extract_letter_id(link: str) -> Optional[int]:
    if not isinstance(link, str):
        return None
    m = re.search(r"/letters/view/(\d+)", link)
    return int(m.group(1)) if m else None

df["letter_id"] = df["link"].map(extract_letter_id)
df["link_canonical"] = df["letter_id"].apply(
    lambda x: f"{BASE_CANONICAL}{int(x)}?left=text" if pd.notnull(x) else pd.NA
)

df[["letter_id", "link", "link_canonical"]].head(10)

,letter_id,link,link_canonical
0,1,https://briefe-der-romantik.de/letters/view/1?...,https://briefe-der-romantik.de/letters/view/1?...
1,2,https://briefe-der-romantik.de/letters/view/2?...,https://briefe-der-romantik.de/letters/view/2?...
2,3,https://briefe-der-romantik.de/letters/view/3?...,https://briefe-der-romantik.de/letters/view/3?...
3,4,https://briefe-der-romantik.de/letters/view/4?...,https://briefe-der-romantik.de/letters/view/4?...
4,5,https://briefe-der-romantik.de/letters/view/5?...,https://briefe-der-romantik.de/letters/view/5?...
5,6,https://briefe-der-romantik.de/letters/view/6?...,https://briefe-der-romantik.de/letters/view/6?...
6,7,https://briefe-der-romantik.de/letters/view/7?...,https://briefe-der-romantik.de/letters/view/7?...
7,8,https://briefe-der-romantik.de/letters/view/8?...,https://briefe-der-romantik.de/letters/view/8?...
8,9,https://briefe-der-romantik.de/letters/view/9?...,https://briefe-der-romantik.de/letters/view/9?...
9,10,https://briefe-der-romantik.de/letters/view/10...,https://briefe-der-romantik.de/letters/view/10...


## 3) Datum: deutsches Datum → ISO (`date_iso`)

Dein `Date`-Feld ist menschenlesbar (z.B. „Donnerstag, 6. Januar 1791“). Für Filter/Timelines brauchen wir zusätzlich eine parsebare Variante.

Hier: eine robuste Heuristik, die **Wochentage ignoriert** und Monate über Mapping parst.

In [6]:
MONTHS_DE = {
    "januar": 1, "jan": 1,
    "februar": 2, "feb": 2,
    "maerz": 3, "mrz": 3,  # wir normalisieren ä->ae
    "april": 4, "apr": 4,
    "mai": 5,
    "juni": 6, "jun": 6,
    "juli": 7, "jul": 7,
    "august": 8, "aug": 8,
    "september": 9, "sep": 9, "sept": 9,
    "oktober": 10, "okt": 10,
    "november": 11, "nov": 11,
    "dezember": 12, "dez": 12,
}

def parse_german_date(s: str) -> Optional[pd.Timestamp]:
    """
    Erwartet typischerweise Formen wie:
      'Donnerstag, 6. Januar 1791'
      '6. Januar 1791'
      'Januar 1791' (=> day=1)
      '1791' (=> Jan 1)
    """
    if not isinstance(s, str):
        return None
    s = s.strip()
    if not s or s.lower() == "unknown":
        return None

    # Wochentag entfernen
    s = re.sub(r"^[A-Za-zÄÖÜäöüß]+,\s*", "", s)

    # Normalisieren
    s_norm = s.lower().replace("ä","ae").replace("ö","oe").replace("ü","ue")

    # 1) dd. month yyyy
    m = re.search(r"(\d{1,2})\.\s*([a-zA-ZäöüÄÖÜß]+)\s+(\d{3,4})", s, re.IGNORECASE)
    if m:
        day = int(m.group(1))
        mon_raw = m.group(2).lower().replace("ä","ae").replace("ö","oe").replace("ü","ue")
        year = int(m.group(3))
        month = MONTHS_DE.get(mon_raw)
        if month:
            return pd.Timestamp(year=year, month=month, day=day)

    # 2) month yyyy
    m = re.search(r"([a-zA-ZäöüÄÖÜß]+)\s+(\d{3,4})", s, re.IGNORECASE)
    if m:
        mon_raw = m.group(1).lower().replace("ä","ae").replace("ö","oe").replace("ü","ue")
        year = int(m.group(2))
        month = MONTHS_DE.get(mon_raw)
        if month:
            return pd.Timestamp(year=year, month=month, day=1)

    # 3) yyyy
    m = re.fullmatch(r"(\d{3,4})", s)
    if m:
        year = int(m.group(1))
        return pd.Timestamp(year=year, month=1, day=1)

    return None

df["date_parsed"] = df["Date"].map(parse_german_date)
df["date_iso"] = df["date_parsed"].dt.strftime("%Y-%m-%d")

df[["Date","date_iso"]].head(20)

,Date,date_iso
0,"Donnerstag, 6. Januar 1791",1791-01-06
1,"Mittwoch, 20. September 1797",1797-09-20
2,"Samstag, 26. Mai 1798",1798-05-26
3,"Mittwoch, 31. Oktober 1798",1798-10-31
4,[Mitte August 1801],1801-08-01
5,[ca. 20. August 1801],1801-08-20
6,"Dienstag, 25. August 1801",1801-08-25
7,[25. August 1801],1801-08-25
8,[ca. 28. August 1801],1801-08-28
9,[ca. 30. August 1801],1801-08-30


## 4) Koordinaten & Distanz: Validierung

Ziel: keine „stillen“ NaNs. Wir loggen grob, wie viele Zeilen unvollständig sind, und setzen `distance_km` neu, falls beide Punkte da sind.

In [7]:
def coerce_float(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series, errors="coerce")

for c in ["dispatch_lat","dispatch_lon","destination_lat","destination_lon","distance_km"]:
    if c in df.columns:
        df[c] = coerce_float(df[c])

missing_dispatch = df["dispatch_lat"].isna() | df["dispatch_lon"].isna()
missing_dest = df["destination_lat"].isna() | df["destination_lon"].isna()

print("Rows total:", len(df))
print("Missing dispatch coords:", int(missing_dispatch.sum()))
print("Missing destination coords:", int(missing_dest.sum()))
print("Missing either side:", int((missing_dispatch | missing_dest).sum()))

Rows total: 4388
Missing dispatch coords: 3
Missing destination coords: 0
Missing either side: 3


In [8]:
# Recompute distance_km where possible (haversine)
def haversine_km(lat1, lon1, lat2, lon2) -> float:
    r = 6371.0088
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dl = np.radians(lon2 - lon1)
    a = np.sin(dphi/2)**2 + np.cos(phi1)*np.cos(phi2)*np.sin(dl/2)**2
    return float(2*r*np.arcsin(np.sqrt(a)))

mask_can = (~missing_dispatch) & (~missing_dest)
df.loc[mask_can, "distance_km"] = df.loc[mask_can].apply(
    lambda r: haversine_km(r["dispatch_lat"], r["dispatch_lon"], r["destination_lat"], r["destination_lon"]),
    axis=1
)

df[["distance_km"]].describe()

,distance_km
count,4385.000000
mean,211.927547
std,220.342114
min,0.000000
25%,71.129758
50%,197.758285
75%,298.868641
max,8336.261592


## 5) Orte aggregieren (Hotspots)

Wir bauen eine Ortstabelle, die Dispatch + Destination zusammenführt, um Counts pro Koordinate zu bekommen.

In [9]:
place_rows = []

for _, r in df.iterrows():
    if pd.notnull(r.get("dispatch_lat")) and pd.notnull(r.get("dispatch_lon")):
        place_rows.append({
            "lat": r["dispatch_lat"],
            "lon": r["dispatch_lon"],
            "role": "dispatch",
            "place": r.get("place_dispatch"),
        })
    if pd.notnull(r.get("destination_lat")) and pd.notnull(r.get("destination_lon")):
        place_rows.append({
            "lat": r["destination_lat"],
            "lon": r["destination_lon"],
            "role": "destination",
            "place": r.get("place_destination"),
        })

places = pd.DataFrame(place_rows)

places_agg = (places
    .groupby(["lat","lon","role"], dropna=False)
    .agg(
        count=("place","size"),
        place_mode=("place", lambda s: s.dropna().mode().iloc[0] if len(s.dropna().mode()) else pd.NA),
    )
    .reset_index()
)

places_agg.sort_values("count", ascending=False).head(20)

,lat,lon,role,count,place_mode
210,52.52437,13.41053,destination,1263,Berlin
93,50.92878,11.58990,dispatch,1130,Jena
211,52.52437,13.41053,dispatch,1103,Berlin
92,50.92878,11.58990,destination,940,Jena
115,51.05089,13.73832,dispatch,192,Dresden
203,52.37403,4.88969,destination,188,Amsterdam
155,51.33962,12.37129,destination,165,Leipzig
99,50.98030,11.32903,destination,147,Weimar
202,52.37052,9.73322,dispatch,146,Hannover
216,52.73679,15.22878,destination,143,Landsberg (Warthe)


## 6) Speichern

Parquet ist für lokale Iteration schneller/sicherer als CSV (Typen bleiben erhalten). CSV zusätzlich für Interop.

In [10]:
df = df.sort_values(["letter_id","date_iso"], na_position="last")

df.to_parquet(OUT_MASTER_PARQUET, index=False)
df.to_csv(OUT_MASTER_CSV, index=False)
places_agg.to_parquet(OUT_PLACES_AGG, index=False)

print("Wrote:", OUT_MASTER_PARQUET)
print("Wrote:", OUT_MASTER_CSV)
print("Wrote:", OUT_PLACES_AGG)

Wrote: g:\Meine Ablage\CodingProjekte\RomKorr\RomKorr\data\processed\letters_master.parquet
Wrote: g:\Meine Ablage\CodingProjekte\RomKorr\RomKorr\data\processed\letters_master.csv
Wrote: g:\Meine Ablage\CodingProjekte\RomKorr\RomKorr\data\processed\places_agg.parquet


In [11]:
import pandas as pd
from pathlib import Path

master = Path("data/processed/letters_master.parquet")
df = pd.read_parquet(master)

print("rows:", len(df))
print("unique letter_id:", df["letter_id"].nunique())
print("missing letter_id:", df["letter_id"].isna().sum())

# Koordinaten-Vollständigkeit
need = ["dispatch_lat","dispatch_lon","destination_lat","destination_lon"]
missing_any = df[need].isna().any(axis=1).sum()
print("rows missing any coords:", missing_any)

# Distance
print("missing distance_km:", df["distance_km"].isna().sum())
print(df["distance_km"].describe())

# Date parsing
print("missing date_iso:", df["date_iso"].isna().sum())
print("date range:", df["date_iso"].dropna().min(), "→", df["date_iso"].dropna().max())

# Duplicates by letter_id (sollte i.d.R. 0 sein)
dups = df.duplicated(subset=["letter_id"]).sum()
print("duplicates by letter_id:", dups)

df.head(3)


rows: 4388
unique letter_id: 4388
missing letter_id: 0
rows missing any coords: 3
missing distance_km: 3
count    4385.000000
mean      211.927547
std       220.342114
min         0.000000
25%        71.129758
50%       197.758285
75%       298.868641
max      8336.261592
Name: distance_km, dtype: float64
missing date_iso: 565
date range: 1771-03-01 → 1802-12-31
duplicates by letter_id: 0


,Date,Sender,Recipient,place_dispatch,place_destination,dispatch_geonames_url,destination_geonames_url,link,dispatch_lat,dispatch_lon,destination_lat,destination_lon,distance_km,letter_id,link_canonical,date_parsed,date_iso
0,"Donnerstag, 6. Januar 1791",August Wilhelm von Schlegel,Christian Gottlob Heyne,Göttingen,Göttingen,https://www.geonames.org/2918632,https://www.geonames.org/2918632,https://briefe-der-romantik.de/letters/view/1?...,51.53443,9.93228,51.53443,9.93228,0.000000,1,https://briefe-der-romantik.de/letters/view/1?...,1791-01-06,1791-01-06
1,"Mittwoch, 20. September 1797",August Wilhelm von Schlegel,Christian Gottlob Heyne,Jena,Göttingen,https://www.geonames.org/2895044,https://www.geonames.org/2918632,https://briefe-der-romantik.de/letters/view/2?...,50.92878,11.58990,51.53443,9.93228,133.622388,2,https://briefe-der-romantik.de/letters/view/2?...,1797-09-20,1797-09-20
2,"Samstag, 26. Mai 1798",August Wilhelm von Schlegel,Georg Joachim Göschen,Berlin,Leipzig,https://www.geonames.org/2950159,https://www.geonames.org/2879139,https://briefe-der-romantik.de/letters/view/3?...,52.52437,13.41053,51.33962,12.37129,149.769217,3,https://briefe-der-romantik.de/letters/view/3?...,1798-05-26,1798-05-26
